In [ ]:
import os
import sys
import json
from io import StringIO
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, roc_curve, r2_score,
    precision_recall_curve, average_precision_score
)

In [ ]:
# ---------------------------- config ----------------------------
with open("config.json", "r", encoding="utf-8") as f:
    cfg = json.load(f)

EXCEL_PATH = cfg["excel_path"]
REAL_SHEET = cfg["real_sheet"]
PRED_SHEET = cfg["pred_sheet"]
OUTDIR = cfg["outdir"]
DEFAULT_WEEKS  = cfg["weeks"]
DEFAULT_TOLERANCE = cfg["tolerance"]

RANDOM_SEED = 42

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def savefig(path: str):
    plt.tight_layout()
    plt.savefig(path, dpi=160, bbox_inches="tight")
    plt.close()

def log(msg: str):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

In [ ]:
# ---------------------------- carga ----------------------------
def load_excel_fixed(path: str, real_sheet: str, pred_sheet: str):
    real = pd.read_excel(path, sheet_name=real_sheet)
    pred = pd.read_excel(path, sheet_name=pred_sheet)
    # normalizacion preventiva
    real.columns = [c.strip().lower().replace(" ", "_") for c in real.columns]
    pred.columns = [c.strip().lower().replace(" ", "_") for c in pred.columns]
    # parse fechas
    real['date'] = pd.to_datetime(real['date'], errors="coerce")
    pred['date_requested']  = pd.to_datetime(pred['date_requested'], errors="coerce")
    pred['date_prediction'] = pd.to_datetime(pred['date_prediction'], errors="coerce")
    return real, pred

def check_required_columns(real: pd.DataFrame, pred: pd.DataFrame):
    # variables indispensables de cada dataframe
    must_real = ['id_commodity','date','value']
    must_pred = ['id_commodity','model','date_requested','date_prediction','prediction']
    miss_real = [c for c in must_real if c not in real.columns]
    miss_pred = [c for c in must_pred if c not in pred.columns]
    # verificación que esten todas las columnas necesarias
    if miss_real or miss_pred:
        print("dataset insuficiente: faltan columnas obligatorias.")
        print("faltan en real:", miss_real)
        print("faltan en predicted:", miss_pred)
        return False
    return True

In [ ]:
# ---------------------------- eda básica ----------------------------
def dump_df_info(df: pd.DataFrame, name: str, outdir: str):
    # guarda head/info/describe y chequeos simples de un dataframe
    df.head(20).to_csv(os.path.join(outdir, f"{name}_head.csv"), index=False)
    df.describe(include='all').to_csv(os.path.join(outdir, f"{name}_describe.csv"))
    buf = StringIO(); df.info(buf=buf)
    with open(os.path.join(outdir, f"{name}_info.txt"), "w", encoding="utf-8") as f:
        f.write(buf.getvalue())
    df.isna().sum().to_csv(os.path.join(outdir, f"{name}_nulls.csv"))
    with open(os.path.join(outdir, f"{name}_dups.txt"), "w", encoding="utf-8") as f:
        f.write(f"duplicados_exactos: {int(df.duplicated().sum())}\n")

def basic_counts(real: pd.DataFrame, pred: pd.DataFrame, outdir: str):
    # commodities por tipo (si existe)
    if 'type' in real.columns and 'id_commodity' in real.columns:
        real.groupby('type')['id_commodity'].nunique().sort_values(ascending=False)\
            .to_csv(os.path.join(outdir, "count_commodities_by_type.csv"))
    # modelos disponibles
    if 'model' in pred.columns:
        pred['model'].value_counts().to_csv(os.path.join(outdir, "count_models.csv"))

In [ ]:
# ---------------------------- construcción del dataset ----------------------------
def filter_horizon(pred: pd.DataFrame, weeks: int, tolerance: int):
    pred = pred.copy()
    pred['horizon_days'] = (pred['date_prediction'] - pred['date_requested']).dt.days
    lo = weeks*7 - tolerance
    hi = weeks*7 + tolerance
    # 4 semanas - diferencia numerica para identificar subidas o bajadas
    pred4 = pred[(pred['horizon_days'] >= lo) & (pred['horizon_days'] <= hi)].copy()
    return pred4, lo, hi

def prepare_real_agg(real: pd.DataFrame):
    agg = {'value': 'mean'}
    # verificacion de columnas categoricas
    for cat in ['type','incoterm','origin','publication']:
        if cat in real.columns:
            # si hay duplicados, se toma el primero
            agg[cat] = 'first'
    real_agg = real.groupby(['id_commodity','date'], as_index=False).agg(agg)
    return real_agg

def merge_with_real(pred4: pd.DataFrame, real_agg: pd.DataFrame):
    # left join - se mantienen todas las predicciones
    merged = pred4.merge(
        real_agg,
        left_on=['id_commodity','date_prediction'],
        right_on=['id_commodity','date'],
        how='left',
        suffixes=('','_real')
    )
    return merged

def add_target_and_errors(merged: pd.DataFrame):
    merged = merged.copy()
    # 1 si valor real es > a la prediccion, 0 si el valor fue menor o igual
    merged['direccion']    = (merged['value'] > merged['prediction']).astype(int)
    # + predijo más bajo que la realidad, - predijo más alto que la realidad
    merged['error_signed'] = merged['value'] - merged['prediction']
    # cuánto se equivocó el modelo sin importar la dirección
    merged['error_abs']    = (merged['value'] - merged['prediction']).abs()
    return merged

def clean_critical(merged: pd.DataFrame):
    crit = ['id_commodity','model','date_prediction','prediction','value','direccion']
    merged = merged.loc[merged[crit].notna().all(axis=1)].copy()
    return merged

def build_wide_and_consensus(merged: pd.DataFrame):
    # un commodity por fecha de prediccion
    key_cols = ['id_commodity','date_prediction']
    wide = merged.pivot_table(index=key_cols, columns='model', values='prediction', aggfunc='mean')
    # renombre para mejor entendimiento
    wide.columns = [f"pred_{str(c)}" for c in wide.columns]
    wide = wide.reset_index()
    # features de consenso
    pred_cols = [c for c in wide.columns if c.startswith("pred_")]
    wide['consensus_mean']  = wide[pred_cols].mean(axis=1, skipna=True)
    wide['consensus_std']   = wide[pred_cols].std(axis=1, ddof=0, skipna=True)
    wide['consensus_min']   = wide[pred_cols].min(axis=1, skipna=True)
    wide['consensus_max']   = wide[pred_cols].max(axis=1, skipna=True)
    wide['consensus_range'] = wide['consensus_max'] - wide['consensus_min']
    return wide

def attach_targets_and_cats(merged: pd.DataFrame, wide: pd.DataFrame):
    key_cols = ['id_commodity','date_prediction']
    cat_cols = [c for c in ['type','incoterm','origin','publication'] if c in merged.columns]
    # dirección - etiqueta binaria, value - valor real del precio, horizon_days - diferencia entre fechas
    targets = merged.groupby(key_cols, as_index=False).agg({'direccion':'first','value':'first','horizon_days':'first'})
    for c in cat_cols:
        targets = targets.merge(merged[key_cols+[c]].drop_duplicates(), on=key_cols, how='left')
    # features de predicción (wide + consenso) con el dataset de targets + categorías
    features0 = wide.merge(targets, on=key_cols, how='inner')
    return features0

def add_hist_errors_train_only(merged: pd.DataFrame, features0: pd.DataFrame):
    # percentil 70%
    # todo lo anterior a esa fecha = train, despues = test
    date_cut = features0['date_prediction'].quantile(0.7)
    train_idx = features0['date_prediction'] <= date_cut
    # se queda solo con filas de entrenamiento
    key_cols = ['id_commodity','date_prediction']
    train_keys = features0.loc[train_idx, key_cols]
    train_long = merged.merge(train_keys, on=key_cols, how='inner')
    hist = train_long.groupby(['id_commodity','model'], as_index=False).agg(
        # hist_signed_mean - si subestima o sobreestima, hist_abs_mean - precisión, hist_count - cantidad de observaciones
        hist_signed_mean=('error_signed','mean'),
        hist_abs_mean=('error_abs','mean'),
        hist_count=('error_abs','count')
    )
    # cada fila un id_commodity, cada columna las metricas historicas para cada modelo
    hw = hist.pivot_table(index='id_commodity', columns='model',
                          values=['hist_signed_mean','hist_abs_mean','hist_count'], aggfunc='first')
    hw.columns = [f"{a}_{b}" for a,b in hw.columns]
    hw = hw.reset_index()
    return features0.merge(hw, on='id_commodity', how='left')

In [ ]:
# ---------------------------- r² en test ----------------------------
def r2_on_test(features: pd.DataFrame, wide: pd.DataFrame, test_idx: pd.Series, outdir: str):
    # calcula r² de predicciones numéricas vs 'value' en test; guarda csv por modelo
    out = {}
    key_cols = ['id_commodity','date_prediction']
    # consenso
    if {'consensus_mean','value'}.issubset(features.columns):
        dfc = features.loc[test_idx, key_cols + ['consensus_mean','value']].dropna()
        if not dfc.empty:
            out['r2_consensus_mean'] = float(r2_score(dfc['value'], dfc['consensus_mean']))
    # por modelo
    pred_cols = [c for c in wide.columns if c.startswith('pred_')]
    if pred_cols:
        test_keys = features.loc[test_idx, key_cols].drop_duplicates()
        wtest = test_keys.merge(wide[key_cols + pred_cols], on=key_cols, how='left')\
                         .merge(features[key_cols + ['value']], on=key_cols, how='left')
        rows = []
        for c in pred_cols:
            dfm = wtest[['value', c]].dropna()
            if not dfm.empty:
                r2c = float(r2_score(dfm['value'], dfm[c]))
                out[f"r2_{c}"] = r2c
                rows.append({'model': c, 'r2': r2c})
        if rows:
            pd.DataFrame(rows).sort_values('r2', ascending=False)\
              .to_csv(os.path.join(outdir, "r2_by_model_test.csv"), index=False)
    return out

# ---------------------------- modelado y evaluación ----------------------------
def build_pipeline(X: pd.DataFrame):
    categorical_cols = [c for c in ['type','incoterm','origin','publication'] if c in X.columns]
    numeric_cols = [c for c in X.columns if c not in categorical_cols]
    preprocess = ColumnTransformer(
        transformers=[
            # rellena valores faltantes con la mediana
            ("num", SimpleImputer(strategy="median"), numeric_cols),
            # reemplaza nulos por categoría más frecuente y hace one-hot
            ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                              ("ohe", OneHotEncoder(handle_unknown="ignore"))]), categorical_cols),
        ],
        # ignora columnas que no se hayan listado
        remainder="drop"
    )
    # deja que los árboles crezcan hasta que se cumpla otra condición
    # mínimo 4 muestras para dividir un nodo
    # mínimo 2 muestras en cada hoja
    # hace reproducible el resultado
    # le da más peso a la minoritaria
    rf = RandomForestClassifier(
        n_estimators=400, max_depth=None, min_samples_split=4, min_samples_leaf=2,
        random_state=RANDOM_SEED, n_jobs=-1, class_weight="balanced"
    )
    clf = Pipeline([("prep", preprocess), ("rf", rf)])
    return clf, numeric_cols, categorical_cols

def compute_baselines(wide: pd.DataFrame, merged: pd.DataFrame):
    # líneas base de comparación
    out = {}
    key_cols = ['id_commodity','date_prediction']
    # dataset reducido con un valor real y la dirección (0/1) para cada predicción
    base = merged.groupby(key_cols, as_index=False).agg({'value':'first','direccion':'first'})
    # clase mayoritaria
    maj = int(base['direccion'].value_counts().idxmax())
    out['majority_class'] = float((base['direccion']==maj).mean())
    # mejor modelo individual
    pred_cols = [c for c in wide.columns if c.startswith("pred_")]
    best_acc, best_model = None, None
    tmp = base.merge(wide[key_cols+pred_cols], on=key_cols, how='left')
    for c in pred_cols:
        pred_dir = (tmp['value'] > tmp[c]).astype(int)
        acc = float((pred_dir == tmp['direccion']).mean())
        if best_acc is None or acc > best_acc:
            best_acc, best_model = acc, c
    out['best_single_model_acc'] = best_acc
    out['best_single_model_name'] = best_model
    return out

def evaluate_and_save(clf, X_test, y_test, outdir, y_proba=None, tag="TEST"):
    # medir rendimiento en test
    y_pred = clf.predict(X_test)
    metrics = {
        # proporción de predicciones correctas
        "accuracy":  float(accuracy_score(y_test, y_pred)),
        # de los predichos como 1 (subida), cuántos eran realmente 1
        "precision": float(precision_score(y_test, y_pred, zero_division=0)),
        # de los que eran realmente 1, cuántos detectó el modelo
        "recall":    float(recall_score(y_test, y_pred, zero_division=0)),
        # media entre precision y recall (balance)
        "f1":        float(f1_score(y_test, y_pred, zero_division=0)),
    }
    if y_proba is not None and len(np.unique(y_test))>1:
        # capacidad de ordenar bien positivos vs negativos
        metrics["roc_auc"] = float(roc_auc_score(y_test, y_proba))
    else:
        metrics["roc_auc"] = None
    return metrics, y_pred

def threshold_tuning(y_true, y_proba, outdir, tag="TEST"):
    if y_proba is None:
        return None
    ths = np.linspace(0.05, 0.95, 19)
    best = (None, None)
    pts = []
    # convierte probabilidades a etiquetas binarias y calcula F1
    for t in ths:
        y_hat = (y_proba >= t).astype(int)
        f1 = f1_score(y_true, y_hat, zero_division=0)
        pts.append((float(t), float(f1)))
        if best[1] is None or f1 > best[1]:
            best = (float(t), float(f1))
    df = pd.DataFrame(pts, columns=["threshold","f1"])
    # gráfica
    plt.figure()
    plt.plot(df["threshold"], df["f1"], marker="o")
    plt.xlabel("Umbral"); plt.ylabel("F1"); plt.title("Tuning de umbral (F1)")
    savefig(os.path.join(outdir, f"threshold_f1_{tag.lower()}.png"))
    return {"best_threshold": best[0], "best_f1": best[1]}

def write_report(outdir, context):
    # reporte sencillo en markdown
    md = []
    md.append(f"# Informe — Metamodelo de dirección ({context['weeks']} semanas)")
    md.append("")
    md.append(f"**Archivo**: `{context['excel_path']}`")
    md.append(f"**Hojas**: Real=`{context['real_sheet']}`, Predicted=`{context['pred_sheet']}`")
    md.append(f"**Ventana**: {context['lo_days']}–{context['hi_days']} días")
    md.append(f"**Corte temporal 70/30**: {context['date_cut']}")
    md.append("")
    md.append("## Baselines")
    mc = context.get('baselines', {})
    maj = mc.get('majority_class')
    if maj is not None:
        md.append(f"- Clase mayoritaria (acc): **{maj:.4f}**")
    best_single = mc.get('best_single_model_name')
    best_single_acc = mc.get('best_single_model_acc')
    if best_single is not None and best_single_acc is not None:
        md.append(f"- Mejor modelo individual `{best_single}` (acc): **{best_single_acc:.4f}**")

    md.append("")
    md.append("## Métricas del metamodelo (TEST)")
    for k, v in context.get('metrics', {}).items():
        if v is None:
            md.append(f"- {k}: NA")
        elif isinstance(v, (int, float, np.number)):
            md.append(f"- {k}: **{float(v):.4f}**")
        else:
            md.append(f"- {k}: **{v}**")

    # r² destacados (si existen)
    if context['metrics'].get('r2_consensus') is not None:
        md.append(f"- r² (consenso): **{context['metrics']['r2_consensus']:.4f}**")
    if context['metrics'].get('best_r2_model') is not None and context['metrics'].get('best_r2_value') is not None:
        md.append(f"- mejor r² individual: `{context['metrics']['best_r2_model']}` = **{context['metrics']['best_r2_value']:.4f}**")

    if context.get("threshold"):
        md.append("")
        md.append(f"**Umbral óptimo (F1)**: {context['threshold']['best_threshold']:.2f} → F1={context['threshold']['best_f1']:.4f}")

    md.append("")
    md.append("## Artefactos")
    md.append("- `metrics_summary.json`")
    md.append("- `feature_importances.csv` y `r2_by_model_test.csv`")
    md.append("- `predicciones_test.csv`")
    md.append("- Figuras: `dist_precios_reales.png`, `dist_horizon_days.png`, `feature_importances_top20.png`,")
    md.append("           `cm_test.png`, `roc_test.png` (si aplica), `pr_test.png`, `threshold_f1_test.png`,")
    md.append("           `threshold_metrics.png`, `scatter_consensus_vs_real_test.png`, `lift_curve.png`, `missing_by_model.png`")

    with open(os.path.join(outdir, "reporte_metamodelo.md"), "w", encoding="utf-8") as f:
        f.write("\n".join(md))

In [ ]:
# ---------------------------- gráficas ----------------------------
def plot_horizon_distribution(df, outdir, lo_days=None, hi_days=None):
    # crea y guarda un histograma de los horizontes de predicción (en días)
    plt.figure()
    if 'horizon_days' not in df.columns:
        if {'date_prediction','date_requested'}.issubset(df.columns):
            df = df.copy()
            df['horizon_days'] = (df['date_prediction'] - df['date_requested']).dt.days
        else:
            return
    df['horizon_days'].plot(kind='hist', bins=40, title='Distribución de horizon_days')
    if lo_days is not None and hi_days is not None:
        ymin, ymax = plt.ylim()
        plt.axvspan(lo_days, hi_days, alpha=0.2)
        plt.text(lo_days, ymax*0.95, f"{lo_days}-{hi_days}d", va='top')
    plt.xlabel('Días'); plt.ylabel('Frecuencia')
    savefig(os.path.join(outdir, "dist_horizon_days.png"))

def plot_feature_importances(clf, num_cols, cat_cols, outdir):
    # qué variables usa más el random forest para decidir la dirección
    importances = clf.named_steps['rf'].feature_importances_
    ohe = clf.named_steps['prep'].named_transformers_.get('cat', None)
    cat_names = ohe.named_steps['ohe'].get_feature_names_out(cat_cols).tolist() if cat_cols else []
    feat_names = num_cols + cat_names
    if len(importances) != len(feat_names):
        feat_names = [f"f{i}" for i in range(len(importances))]
    fi = pd.DataFrame({"feature": feat_names, "importance": importances}).sort_values("importance", ascending=False).head(20)
    plt.figure(figsize=(8,6))
    plt.barh(fi["feature"][::-1], fi["importance"][::-1])
    plt.title("Top 20 importancias"); plt.xlabel("Importancia")
    savefig(os.path.join(outdir, "feature_importances_top20.png"))

def plot_roc(y_test, y_proba, outdir):
    # cada punto = desempeño del modelo en un umbral de probabilidad distinto
    if y_proba is None or len(np.unique(y_test)) < 2: return
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.figure()
    plt.plot(fpr, tpr, label='ROC')
    plt.plot([0,1],[0,1],'--')
    plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('Curva ROC')
    plt.legend()
    savefig(os.path.join(outdir, "roc_test.png"))

def plot_confusion(y_test, y_pred, outdir):
    # diagonal principal (TN y TP) = aciertos; fuera de diagonal = errores
    cm = confusion_matrix(y_test, y_pred)
    plt.figure()
    plt.imshow(cm, interpolation='nearest')
    plt.title('Matriz de confusión'); plt.xlabel('Predicho'); plt.ylabel('Real')
    for (i,j),v in np.ndenumerate(cm):
        plt.text(j, i, str(v), ha='center', va='center')
    savefig(os.path.join(outdir, "cm_test.png"))

def plot_pr_curve(y_test, y_proba, outdir):
    # trade-off entre precision y recall a diferentes umbrales
    if y_proba is None or len(np.unique(y_test)) < 2: return
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    ap = average_precision_score(y_test, y_proba)
    plt.figure()
    plt.plot(recall, precision, label=f"AP = {ap:.3f}")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision–Recall")
    plt.legend()
    savefig(os.path.join(outdir, "pr_test.png"))

def plot_threshold_sweep(y_test, y_proba, outdir):
    # barrido de umbral para ver F1, precision y recall
    if y_proba is None: return
    ths = np.linspace(0.05, 0.95, 19)
    rows = []
    for t in ths:
        y_hat = (y_proba >= t).astype(int)
        rows.append({
            "threshold": t,
            "f1": f1_score(y_test, y_hat, zero_division=0),
            "precision": precision_score(y_test, y_hat, zero_division=0),
            "recall": recall_score(y_test, y_hat, zero_division=0)
        })
    df = pd.DataFrame(rows)
    plt.figure()
    plt.plot(df["threshold"], df["f1"], marker="o", label="F1")
    plt.plot(df["threshold"], df["precision"], marker="o", label="Precision")
    plt.plot(df["threshold"], df["recall"], marker="o", label="Recall")
    plt.xlabel("Umbral"); plt.ylabel("Métrica"); plt.title("Barrido de umbral")
    plt.legend()
    savefig(os.path.join(outdir, "threshold_metrics.png"))

def plot_timeseries_one_commodity(real, wide, commodity_id, outdir):
    # evolución de precios reales, predicciones de modelos, consenso promedio
    if 'id_commodity' not in real.columns or 'date' not in real.columns or 'value' not in real.columns:
        return
    df_real = real[real['id_commodity'] == commodity_id].sort_values('date')
    if df_real.empty: return
    plt.figure(figsize=(9,4))
    plt.plot(df_real['date'], df_real['value'], label='Real')
    if {'id_commodity','date_prediction'}.issubset(wide.columns):
        w = wide[wide['id_commodity']==commodity_id].copy()
        if not w.empty:
            pred_cols = [c for c in w.columns if c.startswith('pred_')]
            w = w.sort_values('date_prediction')
            for c in pred_cols[:4]:
                plt.plot(w['date_prediction'], w[c], label=c)
            if 'consensus_mean' in w.columns:
                plt.plot(w['date_prediction'], w['consensus_mean'], linewidth=2, label='consensus_mean')
    plt.title(f"Serie temporal — id_commodity={commodity_id}")
    plt.xlabel('Fecha'); plt.ylabel('Precio'); plt.legend()
    savefig(os.path.join(outdir, f"ts_{commodity_id}.png"))

def plot_scatter_consensus_vs_real(features, test_idx, outdir):
    # si los puntos están alineados a la diagonal, el consenso predice bien los valores reales
    need = {'consensus_mean','value'}
    if not need.issubset(features.columns): return
    df = features.loc[test_idx, ['consensus_mean','value']].dropna()
    if df.empty: return
    plt.figure()
    plt.scatter(df['consensus_mean'], df['value'], s=14)
    lims = [min(df.min())*0.95, max(df.max())*1.05]
    plt.plot(lims, lims, '--')
    plt.xlabel('Consenso (media predicciones)'); plt.ylabel('Valor real')
    plt.title('Consenso vs Real (TEST)')
    savefig(os.path.join(outdir, "scatter_consensus_vs_real_test.png"))

def plot_lift_curve(y_test, y_proba, outdir):
    # mientras más arriba esté la curva respecto a la línea de 1, más útil es el modelo
    if y_proba is None: return
    df = pd.DataFrame({"y": y_test.values, "p": y_proba}).sort_values("p", ascending=False).reset_index(drop=True)
    df["cum_pos"] = df["y"].cumsum()
    total_pos = df["y"].sum()
    perc = np.arange(1, len(df)+1)/len(df)
    lift = df["cum_pos"] / (total_pos * perc + 1e-9)
    plt.figure()
    plt.plot(perc*100, lift)
    plt.axhline(1.0, linestyle='--')
    plt.xlabel('% de población (ordenada por prob.)'); plt.ylabel('Lift'); plt.title('Lift curve')
    savefig(os.path.join(outdir, "lift_curve.png"))

def plot_missing_by_model(wide, outdir):
    # qué tan incompletas están las predicciones por modelo
    pred_cols = [c for c in wide.columns if c.startswith("pred_")]
    if not pred_cols: return
    miss = wide[pred_cols].isna().mean().sort_values(ascending=False)
    plt.figure(figsize=(8,5))
    plt.barh(miss.index[::-1], miss.values[::-1])
    plt.xlabel('Proporción de NaN'); plt.title('Faltantes por modelo (pred_*)')
    savefig(os.path.join(outdir, "missing_by_model.png"))

In [ ]:
# ---------------------------- main ----------------------------
def main():

    # asegurar que exista el directorio de salida
    ensure_dir(OUTDIR)
    np.random.seed(RANDOM_SEED)

    # 1) carga de hojas excel (real y pred)
    log("cargando excel…")
    real, pred = load_excel_fixed(EXCEL_PATH, REAL_SHEET, PRED_SHEET)

    # inspección básica y conteos
    dump_df_info(real, "real", OUTDIR)
    dump_df_info(pred, "predicted", OUTDIR)
    basic_counts(real, pred, OUTDIR)

    # validación mínima de columnas obligatorias
    if not check_required_columns(real, pred):
        sys.exit(0)

    # 2) filtrar predicciones al horizonte de 4 semanas
    pred4, lo_days, hi_days = filter_horizon(pred, DEFAULT_WEEKS, DEFAULT_TOLERANCE)
    if pred4.empty:
        print("dataset insuficiente: sin cobertura para el horizonte seleccionado.")
        sys.exit(0)

    # 3) merge con precios reales y creación de variables target y error
    real_agg = prepare_real_agg(real)
    merged = merge_with_real(pred4, real_agg)
    merged = add_target_and_errors(merged)
    merged = clean_critical(merged)
    if merged.empty:
        print("dataset insuficiente: después de limpiar críticos quedó vacío.")
        sys.exit(0)

    # 4) crear dataset wide + consenso + targets + categorías + errores históricos
    wide = build_wide_and_consensus(merged)
    features0 = attach_targets_and_cats(merged, wide)
    if features0.empty or 'date_prediction' not in features0.columns or 'direccion' not in features0.columns:
        print("dataset insuficiente: revisa el contenido de las hojas.")
        sys.exit(0)
    features = add_hist_errors_train_only(merged, features0)

    # 5) dividir datos en train y test según corte temporal (70/30)
    date_cut = features['date_prediction'].quantile(0.7)
    train_idx = features['date_prediction'] <= date_cut
    test_idx  = features['date_prediction'] > date_cut

    # 6) calcular baselines (clase mayoritaria y mejor modelo individual)
    baselines = compute_baselines(wide, merged)

    # 7) entrenamiento del modelo random forest con pipeline de preprocesamiento
    y = features['direccion'].astype(int)
    X = features.drop(columns=['direccion','date_prediction','id_commodity'], errors='ignore')
    clf, num_cols, cat_cols = build_pipeline(X)
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_test,  y_test  = X.loc[test_idx],  y.loc[test_idx]
    if len(y_train)==0 or len(y_test)==0:
        print("dataset insuficiente: split temporal dejó folds vacíos.")
        sys.exit(0)

    log("entrenando modelo…")
    clf.fit(X_train, y_train)

    # probabilidad de predicción para la clase positiva
    try:
        y_proba = clf.predict_proba(X_test)[:,1]
    except Exception:
        y_proba = None

    # evaluación del modelo en test y ajuste de umbral
    metrics, y_pred = evaluate_and_save(clf, X_test, y_test, OUTDIR, y_proba=y_proba, tag="TEST")
    tuning = threshold_tuning(y_test, y_proba, OUTDIR, tag="TEST") if y_proba is not None else None

    # r² en test (consenso y por modelo)
    r2_dict = r2_on_test(features, wide, test_idx, OUTDIR)
    if r2_dict:
        if 'r2_consensus_mean' in r2_dict:
            metrics['r2_consensus'] = r2_dict['r2_consensus_mean']
        best_r2 = None; best_name = None
        for k,v in r2_dict.items():
            if k.startswith('r2_pred_'):
                if best_r2 is None or v > best_r2:
                    best_r2, best_name = v, k.replace('r2_','')
        if best_r2 is not None:
            metrics['best_r2_model'] = best_name
            metrics['best_r2_value'] = best_r2

    # exportar importancias de variables a csv
    importances = clf.named_steps['rf'].feature_importances_
    ohe = clf.named_steps['prep'].named_transformers_.get('cat', None)
    if ohe is not None and hasattr(ohe, "named_steps") and "ohe" in ohe.named_steps:
        cat_names = ohe.named_steps['ohe'].get_feature_names_out(cat_cols).tolist()
    else:
        cat_names = []
    feat_names = num_cols + cat_names
    if len(importances) != len(feat_names):
        feat_names = [f"f{i}" for i in range(len(importances))]
    fi = pd.DataFrame({"feature": feat_names, "importance": importances}).sort_values("importance", ascending=False)
    fi.to_csv(os.path.join(OUTDIR, "feature_importances.csv"), index=False)

    # eda mínima: distribución de valores reales
    if 'value' in real.columns and np.isfinite(real['value']).any():
        plt.figure()
        real['value'].plot(kind='hist', bins=40, title='distribución de precios reales')
        plt.xlabel('precio'); plt.ylabel('frecuencia')
        savefig(os.path.join(OUTDIR, "dist_precios_reales.png"))

    # exportar tabla de predicciones en test
    key_cols = [c for c in ['id_commodity','date_prediction','type','incoterm','origin','publication'] if c in features.columns]
    pred_table = features.loc[test_idx, key_cols].copy()
    pred_table['y_true'] = y_test.values
    pred_table['y_pred'] = (y_proba >= 0.5).astype(int) if y_proba is not None else y_pred
    if y_proba is not None:
        pred_table['proba_subida'] = y_proba
    pred_cols = [c for c in wide.columns if c.startswith("pred_")]
    pred_table = pred_table.merge(wide[['id_commodity','date_prediction']+pred_cols], on=['id_commodity','date_prediction'], how='left')
    pred_table.to_csv(os.path.join(OUTDIR, "predicciones_test.csv"), index=False)

    # 8) gráficas principales
    plot_horizon_distribution(pred4, OUTDIR, lo_days, hi_days)
    plot_feature_importances(clf, num_cols, cat_cols, OUTDIR)
    plot_roc(y_test, y_proba, OUTDIR)
    plot_confusion(y_test, y_pred, OUTDIR)
    plot_pr_curve(y_test, y_proba, OUTDIR)
    plot_threshold_sweep(y_test, y_proba, OUTDIR)
    if 'id_commodity' in real.columns and not real['id_commodity'].empty:
        top_id = real['id_commodity'].value_counts().index[0]
        plot_timeseries_one_commodity(real, wide, top_id, OUTDIR)
    plot_scatter_consensus_vs_real(features, test_idx, OUTDIR)
    plot_lift_curve(y_test, y_proba, OUTDIR)
    plot_missing_by_model(wide, OUTDIR)

    # 9) guardar resumen y reporte en json y markdown
    summary = {
        "excel_path": EXCEL_PATH,
        "real_sheet": REAL_SHEET,
        "pred_sheet": PRED_SHEET,
        "weeks": DEFAULT_WEEKS,
        "lo_days": lo_days, "hi_days": hi_days,
        "date_cut": pd.to_datetime(date_cut).strftime("%Y-%m-%d"),
        "n_train": int(len(y_train)), "n_test": int(len(y_test)),
        "metrics": metrics,
        "baselines": baselines,
        "threshold": tuning,
        "r2": r2_dict
    }
    with open(os.path.join(OUTDIR, "metrics_summary.json"), "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2)
    write_report(OUTDIR, summary)

    log("finalizado")
    log(f"métricas test: {metrics}")
    if tuning: log(f"umbral óptimo (f1): {tuning}")

if __name__ == "__main__":
    main()
